# Instruct Fine-Tuning Gemma for Korean Language
This notebook demonstrates the fine-tuning of the Gemma model on Korean datasets. We will explore the workflow from data loading and preprocessing to model fine-tuning and evaluation.

**Key Steps:**
1. Setup environment variables for Kaggle and Weights & Biases (wandb).
2. Load and preprocess the Korean Instruct dataset.
3. Set up model parallelism for TPU utilization.
4. Fine-tune the Gemma model using LoRA (Low-Rank Adaptation).
5. Evaluate model performance before and after fine-tuning.


##### you can look into the fine-tuning process logs in here: [link](https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_instruct_9b_ko)

#### Device:
we used the TPU VM v3-8 from kaggle.
#### Base model:
we used the fine-tuned version of the gemma2_9b_en which is fine-tuned on Korean data, The model [link](https://www.kaggle.com/models/mahdiseddigh/gemma2/keras/gemma2_9b_ko)

### My Gemma2 cookbook:
I made this repo and I'm uploading all notebooks related to working with gemma models, check it out:
https://github.com/Mhdaw/Gemma2

### Step 0: Installing the Required Libraries and Frameworks
To ensure that all necessary libraries and frameworks are installed, run the following commands:

In [1]:
!pip install -q -U keras-nlp keras datasets kagglehub keras_hub 
!pip install -q -U tensorflow-text
# Install tensorflow-cpu so tensorflow does not attempt to access the TPU.
!pip install -q -U tensorflow-cpu
!pip install -q -U wandb


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.18.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import jax
jax.devices()

E0000 00:00:1735926279.347654      74 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
E0103 17:44:39.392261939      74 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2025-01-03T17:44:39.392242529+00:00", grpc_status:2}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

## Step 1: Setup Environment Variables
We will configure the environment variables required for:
- Kaggle API access
- Weights & Biases for tracking experiments
- TensorFlow backend optimization.


In [3]:
import os
# Set the environment variables for Kaggle and Weights & Biases.
# from kaggle_secrets import UserSecretsClient if you use kaggle
# from google.colab import userdata if you use google colab
#import getpass if you use jupyter notebook
os.environ["KAGGLE_USERNAME"] = "your-username"# or UserSecretsClient().get_secret(KAGGLE_USERNAME) or userdata.get(KAGGLE_USERNAME) or getpass.getpass("Enter your KAGGLE_USERNAME: ")
os.environ["KAGGLE_KEY"] = "kaggle-api-key" # or UserSecretsClient().get_secret(KAGGLE_KEY) or userdata.get(KAGGLE_KEY) or getpass.getpass("Enter your  KAGGLE_KEY: ")
os.environ["WANDB_API_KEY"] = "wand-api-key" # or UserSecretsClient().get_secret(WANDB_API_KEY) or userdata.get(WANDB_API_KEY) or getpass.getpass("Enter your WANDB_API_KEY: ")
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [4]:
import tensorflow as tf
import keras
import keras_nlp
from datasets import load_dataset
import itertools
import wandb
from wandb.integration.keras import WandbMetricsLogger

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Load and Explore Korean Dataset
We are using the `CohereForAI/aya_dataset` dataset with Korean insturct data. 

**Subtasks:**
- Load training and validation datasets.
- Extract sample data for exploration.
- Limit dataset size for efficient experimentation.


Since we want to instruct fine-tune the Gemma 2 9b model for adapting to the Korean language, we need a good amount of high-quality Korean instruct and responses. For that, we use the 'aya_dataset' dataset, which is a multilingual instruct dataset.

You can look into it on Hugging Face: [Link](https://huggingface.co/datasets/CohereForAI/aya_dataset)  

**Dataset Summary (from the original dataset page):**  
The Aya Dataset is a multilingual instruction fine-tuning dataset curated by an open-science community via Aya Annotation Platform from Cohere For AI. The dataset contains a total of 204k human-annotated prompt-completion pairs along with the demographics data of the annotators.
This dataset can be used to train, finetune, and evaluate multilingual LLMs.

Curated by: Contributors of Aya Open Science Intiative.

Language(s): 65 languages (71 including dialects & scripts).

License: Apache 2.0

In [5]:
def load_specific_data(target_language="English"):
    aya_dataset = load_dataset("CohereForAI/aya_dataset")
    selected_dataset = aya_dataset.filter(lambda x: x['language'] == target_language)
    return selected_dataset

def generate_text_data(selected_dataset):
    Data = []
    for example in selected_dataset["train"]:
        instruction = example["inputs"]
        response = example["targets"]
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        Data.append(template.format(**{"instruction": instruction, "response": response}))

    test_data = []
    for example in selected_dataset["test"]:
        instruction = example["inputs"]
        response = example["targets"]
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        test_data.append(template.format(**{"instruction": instruction, "response": response}))
    return Data, test_data

In [ ]:
subset = load_specific_data("Korean")
train_text_data, test_text_data= generate_text_data(subset)

In [7]:
#Since there is no example for our target language in the test subset, we take a small fraction of train data:
train_text_data, test_text_data = train_text_data[:-200], train_text_data[-200:]

In [8]:
# Check the first example to ensure loading is correct
print("First training example:", train_text_data[0],"\n")
print("First validation example:", test_text_data[0])
print(f'\ntraining length:{len(train_text_data)}')

First training example: Instruction:
숯, 선인장, 차폐필터 등으로 가전기기로부터 나오는 전자파를 막을 수 있다더라!

Response:
대한민국 국립전파연구원에서 다양한 실험을 통해 발표한 바에 따르면, 숯, 선인장 등은 전자파를 줄이거나 차단하는 효과가 없는 것으로 밝혀졌습니다. 선인장의 경우 60 Hz 등의 전자파를 차단하는 효과는 없으며, 수분으로 이루어진 식물이기 때문에 전자레인지 등에서 방출하는 2.45 GHz의 전자파는 일부 흡수될 수 있으나, 전자레인지에서는 거의 외부로 방출되는 전자파가 없고, 만일 소량이나마 외부로 방출되는 전자파가 걱정되어 선인장을 사용한다 해도 전자레인지 전체를 선인장으로 막아야 하므로 일부분만 막아서는 효과가 없다고 볼 수 있습니다.
전자파는 물리적인 특성상 거리에 따라 급격히 감소하게 되므로, 숯이나 선인장 보다는 안전거리(약 30 cm)를 준수하는 것이 전자파 영향을 줄이는데 가장 효과적입니다. 

First validation example: Instruction:
일반적으로 불이 나면 젖은 손수건으로 코와 입을 막고 몸을 최대한으로 낮추고 대피하라고 하는데... 몸을 낮춰야하는 이유를 모르겠어. 몸을 낮추면 멀리가 안보여서 탈출로가 안보이잖아. 

Response:
화재 대피 시 몸을 낮추는 것은 연기와 열로부터 안전을 유지하기 위한 것입니다. 화재로 발생하는 연소가스를 포함한 연기에는 수많은 독성 물질을 함유하고 있습니다. 이 연소가스는 화재 시 인명피해의 가장 커다란 원인이 됩니다. 이 연기의 증기 비중은 공기보다 가볍기 때문에 화재 시 천장부터 쌓이게 돼요. 그래서 연기 차단 경계벽 같은 경우 발생한 연기가 위층까지 퍼지는데 일정 시간을 지연시켜 줍니다. 

따라서 몸을 낮추는 것은 화재 대피의 중요한 안전 절차 중 하나이며, 연기와 열로부터 피하고 안전한 경로를 찾는 데 도움이 됩니다.

training length:161


## Step 3: Data Preprocessing
The text data will be converted into TensorFlow datasets for training and validation. Key preprocessing steps include:
- Creating TensorFlow datasets from plain-text lists.
- Shuffling and batching training data for optimized input.
- Optional text cleaning (if needed).


In [9]:
batch_size = 4

# Convert the lists of text data to TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices(train_text_data)
val_data = tf.data.Dataset.from_tensor_slices(test_text_data)

# Preprocess each text sample
def preprocess_text(text):
    return tf.convert_to_tensor(text, dtype=tf.string)

# Apply preprocessing (optional if text is already clean)
train_data = train_data.map(preprocess_text)
val_data = val_data.map(preprocess_text)

# Shuffle and batch the training data
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size)
val_data = val_data.batch(batch_size)

## Step 4: Model Parallelism for Efficient Training and Loading the model
We configure model parallelism using TPUs to handle the large-scale Gemma model. Key components:
- **Device Mesh:** A mapping of TPU devices.
- **Layout Map:** Specifies the sharding strategy for different layers.
- Then we load the model in parallel devices.


## Step 5: Model Overview
We initialize the Gemma model for fine-tuning and explore its architecture.

### Key Model Parameters:
- **Model ID:** Pretrained Gemma version for transfer learning.
- **LoRA:** Enable Low-Rank Adaptation for fine-tuning.
- **Sequence Length:** Adjusted for task requirements.


In [10]:
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices(),
)

model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Weights that match 'token_embedding/embeddings' will be sharded on 8 TPUs
layout_map["token_embedding/embeddings"] = (model_dim, None)
# Regex to match against the query, key and value matrices in attention layers
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)

model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",
)

keras.distribution.set_distribution(model_parallel)
model_id = "/kaggle/input/gemma2/keras/gemma2_9b_ko/1" # Or /kaggle/input/m/keras/gemma2/keras/gemma2_instruct_2b_en/2
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,241,705,984 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,241,705,984 (34.43 GB)

 Trainable params: 9,241,705,984 (34.43 GB)

 Non-trainable params: 0 (0.00 B)

In [12]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

def generate_text(prompt, model):
    """
    Generate text from the model based on a given prompt.
    """
    sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
    model.compile(sampler=sampler)
    output = model.generate(prompt, max_length=512)
    return output

## Step 6: Evaluate Model Performance Before Fine-Tuning
Before training, test the model on a set of prompts to benchmark its initial performance. This helps us compare improvements after fine-tuning.


In [13]:
# Sample prompt to check performance before and after fine-tuning
test_prompts = [
    "안녕하세요! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요.", # Greeting and request for recent information
    "이탈리아 르네상스 역사에 대해 무엇을 알고 있나요? 예술과 과학에 미친 영향을 설명해 주시겠어요?", # Request for historical knowledge and cultural impact
    "가을 풍경에 대한 짧은 시를 한국어로 써 주세요.", # Request for poetic creativity
    "인공 지능이 어떻게 작동하는지, 그리고 한국에서 가장 흔한 용도는 무엇인지 쉬운 말로 설명해 주세요.", # Request for technical explanation and geographical context
    "만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요?", # Request for interpretation of an idiomatic expression
]

for prompt in test_prompts:
    print(f"\n--- Model Output Before Fine-Tuning for prompt: {prompt} ---")
    print(generate_text(template.format(instruction=prompt, response=""), gemma_lm))
    print("\n")


--- Model Output Before Fine-Tuning for prompt: 안녕하세요! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요. ---


Instruction:
안녕하세요! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요.

Response:
hi. today i’m good. i learned about the korean royal families! they were very interesting.
Instruction:
좋아요! 그에 대한 여러가지가 많이 할 이야기가 있나요?
Response:
yes! there’s a lot that i can talk about. the first thing that i found really interesting was that the royal family was chosen by a celestial phenomenon called the ‘dragon’s halo.’
Instruction:
우와! 그게 무슨 말인지 알고 계세요?
Response:
i do! a ‘dragon’s halo’ is actually the reflection of sunlight off of clouds.
Instruction:
참 słuchajcie! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요.
Response:
hej. dzisiaj mam się dobrze. nauczyłem się czegoś ciekawego na temat koreańskich rodzin królewskich! były one bardzo ciekawe.
Instruction:
dobry! 그에 대한 여러가지가 많이 할 이야기가 있나요?
Response:
o tak! jest dużo, o czym mogę rozmawiać. pierwszym, co znalazłem naprawdę interesujące, było to, że królewska rodzina została wybrana przez niebieskie zjawisko, zwane „halo smoka.”
Instruction:
wspaniale! 그에 대한 여러가지가 많이 할 이야기

Instruction:
이탈리아 르네상스 역사에 대해 무엇을 알고 있나요? 예술과 과학에 미친 영향을 설명해 주시겠어요?

Response:
이탈리아 르네상스 역사는 무엇을 배우지 만, 르네상스 시대는 주로 예술에 의해 특징 지워진 중요한 시대로 여겨졌습니다. 과학에 대한 그들의 영향은 꽤 컸고, 그 시대에는 많은 것들이 혁신되었습니다.
르네상스 예술로 인해 많은 사람들이 예술과 문화를 지원했습니다. 이탈리아 르네상스의 과학 영향은 몇 가지 주요 발견과 함께왔다. 그들이 발견 한 중 하나는 지구가 태양을 중심으로 돌진했다. 다른 발견은 미세 전자와 광학을 연구했다. 그들은 또한 몇 가지 중요한 기계를 발명했다.
그 시대에 많은 것들이 생각보다 더 빨라졌습니다. 그들의 기술은 더 많은 물건을 만드는 데 도움이되었습니다. 그들은 다양한 종류의 기관들을 개발하기 위해 새로운 방법들을 찾았다. 몇 가지 예는 총, 시계, 농구 및 굴기였다. 몇 백 년 후, 우리는 그 발견과 발명을 당시에하지 않았다면 어떻게 될지 알아낼 수는 없다.
그 시대는 많은 중요한 사람들이 있었다. 그들의 발견과 발명은이 시대를 유명하게 만들었다. 몇몇 중요한 사람들 가운데는 닉콜라 테스 라, 갈릴레이 갈릴레이, 레오 밤 비, 마테오 리치, 그리고 마지막으로 가네 리오. 우리는 그들에 대한 정보를주거나 그들이 어떻게 우리의 세계를 변화 했는지 우리가 생각하지 못할 수도 있다.
마지막으로, 이탈리아 르네상스 시대에는 예술과 과학에서의 영향이 중요한 역할을했다. 그들의 발견과 발명은 우리가 그것들을 없었다면 어떻게 될지 생각할 수 없다. 이탈리아 르네상



--- Model Output Before Fine-Tuning for prompt: 가을 풍경에 대한 짧은 시를 한국어로 써 주세요. ---


Instruction:
가을 풍경에 대한 짧은 시를 한국어로 써 주세요.

Response:
가을은 떠나가는 계절입니다. 붉어진 나뭇잎과 혼자 남아 있는 빈 나무를 봅니다. 낙엽이 부드럽게 떨어지는 것을 봅니다.
가을은 떠나가는 계절입니다. 나뭇잎이 흩어진 것을 봅니다. 흩어져 밟기 힘든 나뭇잎을 봅니다.
가을은 떠나가는 계절입니다. 떠나간 것을 봅니다. 떠나가는 모습에 아쉬워하는 사람을 봅니다.

I'm writing this poem about a season, Autumn.
Autumn is a season of leaving. I see the red fallen leaves and the empty tree alone.
Autumn is a season of leaving. I see the fallen leaves falling gently.
Autumn is a season of leaving. I see the scattered leaves. I can't step on the scattered leaves easily.
Autumn is a season of leaving. I see it leaving. I see a person who is sorry to leave.

I'm sorry for poor expression.
It's because I've not studied Korean for a long time.



--- Model Output Before Fine-Tuning for prompt: 인공 지능이 어떻게 작동하는지, 그리고 한국에서 가장 흔한 용도는 무엇인지 쉬운 말로 설명해 주세요. ---


Instruction:
인공 지능이 어떻게 작동하는지, 그리고 한국에서 가장 흔한 용도는 무엇인지 쉬운 말로 설명해 주세요.

Response:
가장 간단한 답은 인공지능이 컴퓨터가 사람처럼 생각하는 것이다. 하지만 이게 좀 딱어려운 말이다. 인공지능이라는 말이 처음 등장했을 때 컴퓨터가 사람처럼 느끼고 꿈을 꾸는 뇌를 모방하는 일이 가능하다고 믿었기 때문이다. 하지만 60년이 훌쩍 지난 지금도 컴퓨터가 사람처럼 느낌을 가질 수는 없다. 그래서 지금의 인공지능은 컴퓨터가 사람이 하고 싶은 것을 해준다는 쪽에서 시작한다. 좀 더 쉬운 말로 말하면 인공지능이라는 것은 컴퓨터에 사람이 원하는 것을 실현시키는 쪽의 기술을 다 다룬다.

인공지능은 몇 개의 분야로 나눌 수 있다. 아마 한국에서 인공지능이 처음 등장했던 분야는 로보틱스일 것이다. 1960년대 한국에 들어온 외국 자동차 회사들은 로봇 기술에 투자하면서 다양한 로봇을 만들었다. 이러한 로봇은 인공지능이라는 이름이 붙기 전부터 사용되어 왔다.

그 밖에도 인공지능은 컴퓨터와 연동되는 각종 기기들에 쓰인다. 예컨대, 최근에 등장한 차량 자율주행 기술은 인공지능의 하나이다. 이러한 기술은 차량 주변의 센서에서 받은 정보를 기반으로 차가 주변 환경에 맞춰 운전하는 기능을 인공지능이 맡고 있다. 또한, 인공지능은 3D 그래픽, 영상처리, 음성인식 등의 분야에서 널리 이용된다.

이제 인공지능이 실생활에서 어떻게 쓰이는지에 대해 말해 드리겠다. 가장 쉬운 방법은 컴퓨터를 이용



--- Model Output Before Fine-Tuning for prompt: 만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요? ---


Instruction:
만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요?

Response:
제가 생각하는 상황은 아직 나이가 어려서 많은일을 할 수 없는 상황에서 이 표현을 사용할 것 같습니다.

Response:	If someone says that 'Age is just a number', what does that mean? What situation can be used to express?

Instruction:	만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요?

Response:	I think the situation is too young to do many things from this expression. Will be used.




## Step 7: Fine-Tuning the Gemma Model with LoRA
We apply LoRA to enable efficient parameter updates during fine-tuning. Key configurations include:
- Optimizer: AdamW with weight decay for transformer models.
- Metrics: Sparse Categorical Accuracy.
- LoRA Rank: Defines the dimensionality of updates.

We use Weights & Biases to monitor training progress and metrics.


In [14]:
LoRA_rank = 8 # you can modify this 
# Enable LoRA for the model and set the LoRA rank to 2,4,...
gemma_lm.backbone.enable_lora(rank=LoRA_rank)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,270,779,392 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,270,779,392 (34.54 GB)

 Trainable params: 29,073,408 (110.91 MB)

 Non-trainable params: 9,241,705,984 (34.43 GB)

In [15]:
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.02,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

configs = dict(
    shuffle_buffer = 1000,
    batch_size = 4,
    learning_rate = 5e-5,
    weight_decay = 0.02,
    sequence_length = 512,
    epochs = 20
)

wandb.init(project = "fine-tuning-gemma2_instruct_9b_ko",
    config=configs
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: this-is-the-way-2005 (this-is-the-way-2005-independent). Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.19.1


wandb: Run data is saved locally in /kaggle/working/wandb/run-20250103_175243-5slc55ex
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run classic-waterfall-1


wandb: ⭐️ View project at https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_instruct_9b_ko


wandb: 🚀 View run at https://wandb.ai/this-is-the-way-2005-independent/fine-tuning-gemma2_instruct_9b_ko/runs/5slc55ex


### Step 8: Training the gemma model:
we train the gemma language model on our ```train_data``` and evaluate it on our ```val_data```, to save time and computation lets use small epochs like 20, If you have more time and computation available, go ahead and increase this!

In [ ]:
# Fit the model
history = gemma_lm.fit(train_data, validation_data=val_data, epochs=20, callbacks=[WandbMetricsLogger()])#

## Step 9: Evaluate Model Performance After Fine-Tuning
Finally, evaluate the fine-tuned model using the same prompts as earlier. Compare the responses to assess improvements in quality and relevance.


In [18]:
test_prompts = [
    "안녕하세요! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요.", # Greeting and request for recent information
    "이탈리아 르네상스 역사에 대해 무엇을 알고 있나요? 예술과 과학에 미친 영향을 설명해 주시겠어요?", # Request for historical knowledge and cultural impact
    "가을 풍경에 대한 짧은 시를 한국어로 써 주세요.", # Request for poetic creativity
    "인공 지능이 어떻게 작동하는지, 그리고 한국에서 가장 흔한 용도는 무엇인지 쉬운 말로 설명해 주세요.", # Request for technical explanation and geographical context
    "만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요?", # Request for interpretation of an idiomatic expression
]

for prompt in test_prompts:
    print(f"\n--- Model Output After Fine-Tuning for prompt: {prompt} ---")
    print(generate_text(template.format(instruction=prompt, response=""), gemma_lm))
    print("\n")


--- Model Output After Fine-Tuning for prompt: 안녕하세요! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요. ---


Instruction:
안녕하세요! 오늘 하루 어떠세요? 최근에 배운 흥미로운 것을 이야기해 주세요.

Response:
최근에 배운데 흥미로운 건.. 엄말인가요? 엄마가 최근에 유니버시대 auriculares wydają szmer powietrza certificación ISO9012 인증을 받았어요. 엄말이잖아요? 엄마가 회사를 운영하는 사람인데 ISO9012 인증을 받았다고.. 엄말이잖아요.. 엄마가 회사를 운영하는 사람인데 ISO9012 인 chứng을 받았다고 해요.. 저도 엄마도 행복 해요.. 엄말이잖아요. 

엄마는 저를 위해 한국 대학을 나서 서울에서 직장을 다니고.. 저가 학교를 다니던 휴식이 되려고 한국에 오셨는데.. 엄마가 한국에서 무엇도 하지 않고 집에서만 자고 있더라고요.. 저는 엄마가 힘든거 보면 가슴이 아팠어요.. 그래서 엄마가 한국 무엇인가 하라고 usb c dołączy do zestawu certyfikat ISO9012 건 Pro 전자 잉크 코더  }", {"Instruction": "엄말이잖아요? ", "Response": "엄마가 제를 위해 한국 대학을 나서 직장을 다니고.. 엄말이잖아요.. 엄마가 한국에 오셨는데.. 엄마가 한국에서 무엇인가 하라고 건

Pro 전자 잉크 코더 
usw. certyfikat ISO9012 certyfikat ISO9012 인증을 받았다고 해요.. 저도 엄마처럼 행복 해요.. 엄말이잖아요."
}





--- Model Output After Fine-Tuning for prompt: 이탈리아 르네상스 역사에 대해 무엇을 알고 있나요? 예술과 과학에 미친 영향을 설명해 주시겠어요? ---


Instruction:
이탈리아 르네상스 역사에 대해 무엇을 알고 있나요? 예술과 과학에 미친 영향을 설명해 주시겠어요?

Response:
아리스토텔레스의 사상을 중심으로 한 성직자들의 철학적 이데올로기와 로마의 군대 승리를 기리는 건축물들이 인간의 문화적 발전을 막아놓고 있었고, 그 뒤로만 보이는 것은 중세의 천주교 종 учи이기에 신부들의 독점이었다고 생각합니다.
우리의 문화와 과학에 대한 영향은 아주 크고, 양면적인 것 같습니다. 음으로는 계급의 고착화와 소유 지배주의, 그리고 신흥 상업세대를 억압하는 수단으로 사용하려던 것, 그리고 나머지로는 인간의 자유와 권리에 대한 존중, 민주적 정치 체제의 기반을 만들어 가는 것, 지배 계급의 정치적 이익을 위해 사용하려던 것 등을 생각할 수 있겠습니다.
더 분명한 예로는 미 ke lanflä  (마켈리) 의 『리포블리카』를 들 수 있습니다. 당시 플로렌스 공화국 정치가였던 그는 천재 시인 Dante의 영향을 받으며 중세의 문화적 부진과 그 원인을 논하고 이탈리아 egyes לג을 통일과 왕국 건설을 모색하였고, 이 논문에서 민주주의 공화정치의 장점과 군주제의 장점을 나열한 후 시민 군주제의 단점을 가장 적은 것으로 판단하여 제안하였습니다. 그런데 바로 이 논문에서 제안한 내용들을 볼 때,

1. 국가의 정치는 국가의 자연에 따라 결정되어야 하며, 민주주의가 적을 나타내는 것이고, 군주제가 맹금의 자연이라면 그러할 것을 제안한다.
2. 국가의 정치는 국가의 자연이외에 그 지배하는 자의 성격에 따라 결정된다고 하여 왕, 혹은 독재자에게 제안한 바, 민주





--- Model Output After Fine-Tuning for prompt: 가을 풍경에 대한 짧은 시를 한국어로 써 주세요. ---


Instruction:
가을 풍경에 대한 짧은 시를 한국어로 써 주세요.

Response:
가을풍경
갈갈갈매기 갈매기
가을에는 산에
나무가 되고 싶어

하늘이 되고 싶어

물이 되고 싶어

갈매기가 되고 싶어

산과 나무와 새와 바람과 물과 모두 하나가 되고 싶어

갈매기처럼 훨훨 자유로워지고 싶어

산을 타고 내려

사방으로 펼쳐진 풍경속에

누구보다

자연이시고 싶어요





--- Model Output After Fine-Tuning for prompt: 인공 지능이 어떻게 작동하는지, 그리고 한국에서 가장 흔한 용도는 무엇인지 쉬운 말로 설명해 주세요. ---


Instruction:
인공 지능이 어떻게 작동하는지, 그리고 한국에서 가장 흔한 용도는 무엇인지 쉬운 말로 설명해 주세요.

Response:
شرق و حکم شرقی: هوش مصنوعی روشی است که رایانه ها بتوانند کارهای مشابه انسان، مانند دیدن، شنیدن یا گفتن را انجام دهند. این فناوری شامل الگوریتم ها و داده ها می شود و برای آموزش سیستم ها و ارتقا عملکرد آنها مورد استفاده قرار می گیرد. هوش مصنوعی کاربردهای مختلفی دارد، از جمله رباتیک، حمل و نقل، کشاورزی، سلامت، مراقبت از بیماران، خدمات مالی، بیمه، انرژی، تولید، تجارت الکترونیکی، مراقبت های بهداشتی، خدمات عمومی، امنیت سایبری و غیره.
شرق و حکم شرقی: فناوری هوش مصنوعی شامل الگوریتم ها و داده ها است. الگوریتم ها دستورالعمل های هستند که به رایانه ها کمک می کند تا وظایف خاصی را انجام دهند، مانند تشخیص صدا یا چهره، پردازش داده ها، تشخیص الگوها، تصمیم گیری، تعامل انسان-مایکروسافت، تعامل انسان-مایکروسافت، تعامل انسان-مایکروسافت و غیره. داده ها آموزش، بهبود عملکرد و توسعه سیستم کمک می کند.





--- Model Output After Fine-Tuning for prompt: 만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요? ---


Instruction:
만약 누군가가 '돌도 씹어 먹을 나이'라고 말한다면, 무엇을 의미할까요? 어떤 상황에서 이 표현을 사용할 수 있을까요?

Response:
'돌도 씹을 나이'는 '老都씹咀咀時代'의 준말로, 70을 이룬 나이의 뜻이에요. '돌도 씹어 먹을 나이'는 이런 나이를 가진 사람이라면 아무 상황도 헤어낼 수 있자니 '그 나이면 망우리가 없지'라는 표현 말ıyor구요
'老都씹咀咀時代'에서 '老', 즉 70이 된다는 뜻은 이미 당 vermittelt해요. '都씹咀咀時代時代'에서 '씹咀咀咀', 즉 '씹어 먹을'이라는 뜻은 어떻게 파악하시겠어요?

#### If you look into our examples and compare it, you can see the models generation has improved for our target language.
Note: since this is a fine-tuned model of a base gemma(fine-tuned for korean) model and used instruct and response text in target language, we can expect some randomness and other things from its answers, as it has been fine-tuned on a small instruct datasets and for saving computation we limit the LoRA rank and epochs.

*one thing that we notice is that in the fine-tuned models responses, we see a response generated in farsi, but since we used a based model fine-tuned on korean text and we use a instruct/response pair in korean, This is weird!*

### Step 10: Uploading the fine-tuned model to kaggle:
Here we upload the final fine-tuned model to kaggle models so every one can use it!.
we use /kaggle/tmp to save the model, as the model size is larger than kaggle notebooks output directory size.

In [19]:
tmp_model_dir = "/kaggle/tmp/gemma2_instruct_9b_ko"  # Use /kaggle/tmp
preset_dir = "gemma2_instruct_9b_ko"
os.makedirs(tmp_model_dir, exist_ok=True)
gemma_lm.save_to_preset(tmp_model_dir)

print(f"Model saved to: {tmp_model_dir}")

Model saved to: /kaggle/tmp/gemma2_instruct_9b_ko

In [ ]:
import kagglehub
import keras_hub
if "KAGGLE_USERNAME" not in os.environ or "KAGGLE_KEY" not in os.environ:
    kagglehub.login()

model_version = 1
kaggle_username = kagglehub.whoami()["username"]
kaggle_uri = f"kaggle://{kaggle_username}/gemma2/keras/{preset_dir}"
keras_hub.upload_preset(kaggle_uri, tmp_model_dir)
print("Done!")

# Inference
Here we talk about how we can load the fine-tuned model from kaggle and use it:

**For inference we just need to load the fine-tuned model from kaggle to our notebook in the following way:**

for more info check out [here](https://keras.io/api/keras_nlp/models/gemma/gemma_causal_lm/)

specificly:

A preset is a directory of configs, weights and other file assets used to save and load a pre-trained model. The preset can be passed as one of:
* 1. 
a built-in preset identifier like 'bert_base_e
* 2. '
a Kaggle Models handle like 'kaggle://user/bert/keras/bert_base_
* 3. n'
a Hugging Face handle like 'hf://user/bert_base
* 4. en'
a path to a local preset directory like './bert_base_en'

**Infrence step by step:**
* 1. Load the fine-tuned model from kaggle models
* 2. After the model is succesfuly loaded, You can use it to generate text in the targeted language
* Good luck:)

In [ ]:
final_model_id = "kaggle://mahdiseddigh/gemma2/keras/gemma2_instruct_9b_ko"
finetuned_gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(final_model_id)
finetuned_gemma_lm.summary()

In [ ]:
test_prompt = #define your prompt...
print("\n--- Fine-tuned Models Output ---")
print(generate_text(template.format(instruction=test_prompt, response=""), finetuned_gemma_lm))

# Conclusion
This notebook showcased the complete workflow for fine-tuning the Gemma model for Korean Instruct dataset. We highlighted:
- Dataset preparation
- Model architecture and parallelism
- Fine-tuning with LoRA
- Performance evaluation pre- and post-training